In [1]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c r-street-quant-challenge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  0% 0.00/2.48M [00:00<?, ?B/s]
100% 2.48M/2.48M [00:00<00:00, 222MB/s]


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers import Dense, Dropout, LSTM

In [ ]:
df=pd.read_csv("/content/train_data.csv")
df2=pd.read_csv("/content/test_data.csv")
df3=pd.read_csv("/content/train_target.csv")

In [ ]:
target=df3['target'].to_numpy()
df3['One']=1
diff=df3['One']-df3['target']
diff=diff.abs()
from sklearn.metrics import roc_auc_score
diff.mean()

0.49011764705882355

In [ ]:
roc_auc_score(df3['target'],df3['One'])

0.5

In [ ]:
df.columns

Index(['Id', 'date', 'open', 'high', 'low', 'close', 'day', 'week', 'SMA_7',
       'SMA_14', 'SMA_21', 'RSI_7', 'RSI_14', 'RSI_21', 'ATR_14', 'bb_mavg',
       'bb_hband', 'bb_lband'],
      dtype='object')

In [ ]:
def adx(DF,n=20):
    "function to calculate ADX"
    df2 = DF.copy()
    df2['H-L']=abs(df2['high']-df2['low'])
    df2['H-PC']=abs(df2['high']-df2['close'].shift(1))
    df2['L-PC']=abs(df2['low']-df2['close'].shift(1))
    df2['TR']=df2[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df2['+DM']=np.where((df2['high']-df2['high'].shift(1))>(df2['low'].shift(1)-df2['low']),df2['high']-df2['high'].shift(1),0)
    df2['+DM']=np.where(df2['+DM']<0,0,df2['+DM'])
    df2['-DM']=np.where((df2['low'].shift(1)-df2['low'])>(df2['high']-df2['high'].shift(1)),df2['low'].shift(1)-df2['low'],0)
    df2['-DM']=np.where(df2['-DM']<0,0,df2['-DM'])

    df2["+DMMA"]=df2['+DM'].ewm(span=n,min_periods=n).mean()
    df2["-DMMA"]=df2['-DM'].ewm(span=n,min_periods=n).mean()
    df2["TRMA"]=df2['TR'].ewm(span=n,min_periods=n).mean()

    df2["+DI"]=100*(df2["+DMMA"]/df2["TRMA"])
    df2["-DI"]=100*(df2["-DMMA"]/df2["TRMA"])
    df2["DX"]=100*(abs(df2["+DI"]-df2["-DI"])/(df2["+DI"]+df2["-DI"]))
    
    df2["ADX"]=df2["DX"].ewm(span=n,min_periods=n).mean()

    return df2


In [ ]:
#Reached 0.5172779405972452
#Reached 0.5172779405972452
dummy=df.copy()
dummy=adx(dummy,19)
dummy['SMA_DI']=0.503
dummy['diff']=dummy['+DI']-dummy['-DI']
dummy['cross']=dummy['SMA_7']-dummy['SMA_21']
k,m=0,0
for i in range(20):
  dummy['SMA_DI'][i]=0.503
for j in range(20,25500):
  if dummy['diff'][j]>0 and dummy['cross'][j]>0:
    dummy['SMA_DI'][j]=1
    k+=1
  elif dummy['diff'][j]<0 and dummy['cross'][j]<0:
    dummy['SMA_DI'][j]=0.05
    m+=1
  else:
    dummy['SMA_DI'][j]=model.predict(inlet)
    if(dummy['diff'][j]<0 and dummy['cross'][j]>0):
      if (dummy['cross'][j]>(-1)*dummy['diff'][j]): dummy['SMA_DI'][j]=1
      else: dummy['SMA_DI'][j]=0
    else:
      if (dummy['diff'][j]>(-1)*dummy['cross'][j]): dummy['SMA_DI'][j]=1
      else:
        dummy['SMA_DI'][j]=model.predict(df_train[j])
a=pd.DataFrame()
a['Id']=np.linspace(0,10316,num=10317)
a['target']=dummy['SMA_DI']

a.to_csv("Submission-1.csv")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


NameError: ignored

In [ ]:
contra=df.copy()
contra['contrarian']=0.5
for i in range(5,25500):
  contra['contrarian'][i]=0 if (df['close'][i]-df['close'][i-5]>0) else 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
def MACD(DF,a=12,b=26,c=9):
    """function to calculate MACD
       typical values a(fast moving average) = 12; 
                      b(slow moving average) =26; 
                      c(signal line ma window) =9"""
    df = DF.copy()
    df["MA_Fast"]=df["close"].ewm(span=a,min_periods=a).mean()
    df["MA_Slow"]=df["close"].ewm(span=b,min_periods=b).mean()
    df["MACD"]=df["MA_Fast"]-df["MA_Slow"]
    df["Signal"]=df["MACD"].ewm(span=c,min_periods=c).mean()
    df.dropna(inplace=True)
    return df

In [ ]:
df=MACD(df)
df2=MACD(df2)

In [ ]:
macd_ind=df.copy()
macd_ind['macd']=0.5
for i in range(99,25500):
  if (macd_ind['macd'][i]>0):
    macd_ind['macd']=1
  else:
    macd_ind['macd']=0

difference=macd_ind['macd']-df3['target']
difference=difference.abs()
loss=difference.mean()
loss
label=df3['target'][33:]
roc_auc_score(label,macd_ind['macd'])

0.5

In [ ]:
difference=contra['contrarian']-df3['target']
difference=difference.abs()
loss=difference.mean()
loss

0.5056666666666667

In [ ]:
loss_min=1
index=0
for i in range(5,100):
  temp=adx(df,i)
  temp["technical wonly"]=0.5
  for i in range(20):
    temp["technical wonly"][i]=0.5
  for j in range(20,25500):  
    if (temp['RSI_14'][j]>60 and temp['+DI'][j]>temp['-DI'][j]):
      temp["technical wonly"][j]=1
    elif temp['RSI_14'][j]<50 and temp['+DI'][j]>temp['-DI'][j]:
      temp["technical wonly"][j]=0
  difference=temp["technical wonly"]-df3['target']
  difference=difference.abs()
  loss=difference.mean()
  if loss<loss_min:
    loss_min=loss
    index=i
    

print(loss_min)
print(i)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


KeyError: ignored

In [ ]:
df=adx(df)
df2=adx(df2)
"""df['RSI_indicator']=df['+DI']
df2['RSI_indicator']=df2['+DI']
df['DI_indicator']=df['+DI']
for i in range(0,25500):
  df['DI_indicator']=df['+DI'][i]-df['-DI'][i]
  if df['RSI_7'][i]>60 and df['+DI'][i]>df['-DI'][i]:
    df['RSI_indicator'][i]=df['RSI_7'][i]-55
  else:
    df['RSI_indicator'][i]=0
  if df['RSI_7'][i]<50 and df['+DI'][i]>df['-DI'][i]:
    df['RSI_indicator'][i]=df['RSI_7'][i]-55
for i in range(0,10316):
  if df2['RSI_7'][i]>60 and df2['+DI'][i]>df2['-DI'][i]:
    df2['RSI_indicator'][i]=1
  else:
    df2['RSI_indicator'][i]=0
  if df2['RSI_7'][i]<50 and df2['+DI'][i]>df2['-DI'][i]:
    df2['RSI_indicator'][i]=-1"""

"df['RSI_indicator']=df['+DI']\ndf2['RSI_indicator']=df2['+DI']\ndf['DI_indicator']=df['+DI']\nfor i in range(0,25500):\n  df['DI_indicator']=df['+DI'][i]-df['-DI'][i]\n  if df['RSI_7'][i]>60 and df['+DI'][i]>df['-DI'][i]:\n    df['RSI_indicator'][i]=df['RSI_7'][i]-55\n  else:\n    df['RSI_indicator'][i]=0\n  if df['RSI_7'][i]<50 and df['+DI'][i]>df['-DI'][i]:\n    df['RSI_indicator'][i]=df['RSI_7'][i]-55\nfor i in range(0,10316):\n  if df2['RSI_7'][i]>60 and df2['+DI'][i]>df2['-DI'][i]:\n    df2['RSI_indicator'][i]=1\n  else:\n    df2['RSI_indicator'][i]=0\n  if df2['RSI_7'][i]<50 and df2['+DI'][i]>df2['-DI'][i]:\n    df2['RSI_indicator'][i]=-1"

In [ ]:
df.columns

Index(['Id', 'date', 'open', 'high', 'low', 'close', 'day', 'week', 'SMA_7',
       'SMA_14', 'SMA_21', 'RSI_7', 'RSI_14', 'RSI_21', 'ATR_14', 'bb_mavg',
       'bb_hband', 'bb_lband', 'MA_Fast', 'MA_Slow', 'MACD', 'Signal', 'H-L',
       'H-PC', 'L-PC', 'TR', '+DM', '-DM', '+DMMA', '-DMMA', 'TRMA', '+DI',
       '-DI', 'DX', 'ADX'],
      dtype='object')

In [ ]:
df['diff']=df['SMA_7']-df['SMA_21']

In [ ]:
df_train=df.loc[75:,[ 'RSI_14', 'RSI_21','diff', 'ATR_14',  'MACD', 'Signal', 'H-L',
       'H-PC', 'L-PC', 'TR', '+DM', '-DM', '+DMMA', '-DMMA', 'TRMA', '+DI',
       '-DI', 'DX', 'ADX']]
df_train
df_label=df3.loc[75:,['target']]
df_train

,RSI_14,RSI_21,diff,ATR_14,MACD,Signal,H-L,H-PC,L-PC,TR,+DM,-DM,+DMMA,-DMMA,TRMA,+DI,-DI,DX,ADX
75,77.327807,71.027528,0.001924,0.003555,0.000507,-0.001038,0.008988,0.020613,0.011625,0.020613,0.020300,0.000000,0.002614,0.000560,0.004082,64.036903,13.713549,64.724195,32.073403
76,75.992460,70.058460,0.003608,0.003584,0.001643,-0.000502,0.003962,0.002644,0.001319,0.003962,0.000000,0.000000,0.002362,0.000506,0.004070,58.026076,12.426326,64.724195,35.492560
77,76.995957,71.031892,0.005357,0.003493,0.002608,0.000120,0.002312,0.001412,0.000900,0.002312,0.000000,0.000088,0.002134,0.000465,0.003901,54.716101,11.933524,64.190274,38.469531
78,72.029972,67.502655,0.007269,0.003440,0.003190,0.000734,0.002744,0.000488,0.002256,0.002744,0.000262,0.000000,0.001954,0.000421,0.003789,51.572792,11.102622,64.571046,41.154370
79,72.116493,67.578217,0.009147,0.003291,0.003617,0.001311,0.001363,0.000631,0.000731,0.001363,0.000000,0.000294,0.001766,0.000409,0.003556,49.676778,11.488424,62.434771,43.326742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25495,53.338774,51.661272,0.001938,0.003345,-0.000450,-0.001063,0.001213,0.001144,0.000069,0.001213,0.001050,0.000000,0.000924,0.000706,0.003231,28.607173,21.860265,13.368835,19.592795
25496,50.340798,49.789358,0.002533,0.003262,-0.000365,-0.000923,0.002181,0.000681,0.001500,0.002181,0.000000,0.000669,0.000836,0.000703,0.003131,26.709170,22.444037,8.677223,18.553216
25497,49.597282,49.324591,0.002079,0.003099,-0.000320,-0.000803,0.000988,0.000631,0.000356,0.000988,0.000000,0.000194,0.000757,0.000654,0.002927,25.850956,22.353303,7.255899,17.477281
25498,50.488807,49.880711,0.002063,0.002942,-0.000251,-0.000692,0.000894,0.000869,0.000025,0.000894,0.000000,0.000000,0.000685,0.000592,0.002733,25.045917,21.657187,7.255899,16.503816


In [ ]:
df_train.shape

(25425, 19)

In [ ]:
model = Sequential([LSTM(256,input_shape=(19,1),activation='relu'),Dense(256,activation='relu'),Dense(128,activation='relu'),Dense(128,activation='relu'),Dense(1)])

model.summary()

model.compile(loss='mean_absolute_error', optimizer='adam',metrics=['mean_squared_error','mean_absolute_percentage_error'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 256)               264192    
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 379,521
Trainable params: 379,521
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(df_train,df_label,epochs=20)

Epoch 1/20
795/795 [==============================] - 80s 95ms/step - loss: 0.5027 - mean_squared_error: 0.3168 - mean_absolute_percentage_error: 256229312.0000
Epoch 2/20
795/795 [==============================] - 63s 80ms/step - loss: 0.4992 - mean_squared_error: 0.2915 - mean_absolute_percentage_error: 299425760.0000
Epoch 3/20
795/795 [==============================] - 65s 82ms/step - loss: 0.4949 - mean_squared_error: 0.3614 - mean_absolute_percentage_error: 404063072.0000
Epoch 4/20
795/795 [==============================] - 64s 81ms/step - loss: 0.4927 - mean_squared_error: 0.3995 - mean_absolute_percentage_error: 434920320.0000
Epoch 5/20
795/795 [==============================] - 58s 73ms/step - loss: 0.4923 - mean_squared_error: 0.4193 - mean_absolute_percentage_error: 449731840.0000
Epoch 6/20
795/795 [==============================] - 66s 83ms/step - loss: 0.4923 - mean_squared_error: 0.4226 - mean_absolute_percentage_error: 450842144.0000
Epoch 7/20
795/795 [==============

In [ ]:
df.columns

Index(['Id', 'date', 'open', 'high', 'low', 'close', 'day', 'week', 'SMA_7',
       'SMA_14', 'SMA_21', 'RSI_7', 'RSI_14', 'RSI_21', 'ATR_14', 'bb_mavg',
       'bb_hband', 'bb_lband', 'H-L', 'H-PC', 'L-PC', 'TR', '+DM', '-DM',
       '+DMMA', '-DMMA', 'TRMA', '+DI', '-DI', 'DX', 'ADX', 'RSI_indicator',
       'DI_indicator'],
      dtype='object')

In [ ]:
label_train=df3.loc[39:25500,['target']]
label_train=label_train.to_numpy()
label_train

array([[1],
       [0],
       [1],
       ...,
       [1],
       [0],
       [0]])

AttributeError: ignored

In [ ]:
df_train=df.loc[39:10317,[
       'RSI_7', 'RSI_14', 'RSI_21', 'ATR_14',  'H-L', 'H-PC', 'L-PC', 'TR', '+DM', '-DM',
       '+DMMA', '-DMMA', 'TRMA', '+DI', '-DI', 'DX', 'ADX', 'RSI_indicator',
       'DI_indicator']]
k=df_train.dropna()
k

,RSI_7,RSI_14,RSI_21,ATR_14,H-L,H-PC,L-PC,TR,+DM,-DM,+DMMA,-DMMA,TRMA,+DI,-DI,DX,ADX,RSI_indicator,DI_indicator
39,70.878090,61.404620,57.528386,0.001594,0.000881,0.000287,0.000594,0.000881,0.000000,0.000244,0.000516,0.000263,0.001471,35.117559,17.854571,32.588813,19.807192,15.878090,22.711733
40,47.521891,52.447363,52.623022,0.001892,0.005769,0.000400,0.005369,0.005769,0.000000,0.004894,0.000466,0.000711,0.001888,24.710484,37.670200,20.775208,19.912223,0.000000,22.711733
41,55.361670,55.631504,54.484659,0.001866,0.001531,0.001413,0.000119,0.001531,0.000000,0.000000,0.000421,0.000642,0.001853,22.737001,34.661700,20.775208,20.004633,0.000000,22.711733
42,44.989121,50.786886,51.664862,0.001869,0.001900,0.000069,0.001831,0.001900,0.000000,0.000919,0.000381,0.000669,0.001858,20.491672,36.013799,27.470130,20.794691,0.000000,22.711733
43,56.257937,55.685012,54.574183,0.001849,0.001587,0.001294,0.000294,0.001587,0.000175,0.000000,0.000361,0.000604,0.001831,19.700783,33.004701,25.241998,21.260408,0.000000,22.711733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10312,57.562220,51.643923,48.979101,0.003817,0.003119,0.002625,0.000494,0.003119,0.002238,0.000000,0.000829,0.000737,0.003633,22.806490,20.281514,5.860046,15.158616,0.000000,22.711733
10313,73.807565,59.775830,54.178170,0.003962,0.005844,0.005406,0.000437,0.005844,0.003087,0.000000,0.001044,0.000667,0.003844,27.154205,17.344954,22.043675,15.814336,18.807565,22.711733
10314,86.667797,71.085792,62.779617,0.004281,0.008419,0.008319,0.000100,0.008419,0.006275,0.000000,0.001542,0.000603,0.004280,36.031350,14.095317,43.761205,18.475942,31.667797,22.711733
10315,87.693093,72.305438,63.816847,0.004247,0.003812,0.002238,0.001575,0.003812,0.001181,0.000000,0.001508,0.000546,0.004235,35.598579,12.886845,46.842393,21.177509,32.693093,22.711733


In [ ]:
import tensorflow as tf


In [ ]:
model=tf.keras.models.load_model("BruteForce.h5")
model.fit(df_train,df_train_label,epochs=45)

Epoch 1/45
172/172 [==============================] - 13s 69ms/step - loss: 0.4941 - mean_squared_error: 0.4595 - mean_absolute_percentage_error: 475448992.0000
Epoch 2/45
172/172 [==============================] - 12s 70ms/step - loss: 0.4935 - mean_squared_error: 0.4674 - mean_absolute_percentage_error: 479747072.0000
Epoch 3/45
172/172 [==============================] - 11s 67ms/step - loss: 0.4941 - mean_squared_error: 0.4673 - mean_absolute_percentage_error: 479762752.0000
Epoch 4/45
172/172 [==============================] - 11s 66ms/step - loss: 0.4936 - mean_squared_error: 0.4749 - mean_absolute_percentage_error: 483744832.0000
Epoch 5/45
172/172 [==============================] - 12s 71ms/step - loss: 0.4939 - mean_squared_error: 0.4618 - mean_absolute_percentage_error: 476727296.0000
Epoch 6/45
172/172 [==============================] - 11s 66ms/step - loss: 0.4940 - mean_squared_error: 0.4622 - mean_absolute_percentage_error: 476730336.0000
Epoch 7/45
172/172 [==============

In [ ]:
b=pd.DataFrame()
b['target']=0
print(b)
k=model.predict(df_train)


Empty DataFrame
Columns: [target]
Index: []


In [ ]:
b['target']=k[0:10316][0]

In [ ]:
k[9]

array([0.9553436], dtype=float32)

In [ ]:
df_train.shape

(25425, 19)

In [ ]:
model.fit(df_train,df_train_label,epochs=45,validation_data=[df_test,df_test_label])

Epoch 1/45
624/624 [==============================] - 44s 71ms/step - loss: 0.4900 - mean_squared_error: 0.4371 - mean_absolute_percentage_error: 459257120.0000 - val_loss: 0.4938 - val_mean_squared_error: 0.4468 - val_mean_absolute_percentage_error: 468754272.0000
Epoch 2/45
624/624 [==============================] - 45s 73ms/step - loss: 0.4907 - mean_squared_error: 0.4424 - mean_absolute_percentage_error: 463660064.0000 - val_loss: 0.4939 - val_mean_squared_error: 0.4924 - val_mean_absolute_percentage_error: 492760320.0000
Epoch 3/45
624/624 [==============================] - 44s 70ms/step - loss: 0.4903 - mean_squared_error: 0.4470 - mean_absolute_percentage_error: 466217088.0000 - val_loss: 0.4938 - val_mean_squared_error: 0.4483 - val_mean_absolute_percentage_error: 469597760.0000
Epoch 4/45
624/624 [==============================] - 45s 71ms/step - loss: 0.4904 - mean_squared_error: 0.4569 - mean_absolute_percentage_error: 472205152.0000 - val_loss: 0.4945 - val_mean_squared_err

In [ ]:
model.sa

In [ ]:
for i in range(39,20000,500):
  df_train_temp=df_train.iloc[i,500+i]
  df_label_temp=df_train_label.iloc[i,500+i]

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
from google.colab import files
pqrs=files.upload()
model=tf.keras.models.load_model('Windowed.h5')

Saving Windowed.h5 to Windowed.h5


In [ ]:
from sklearn.model_selection import train_test_split
for i in range(39,25500,500):
  df_train=df.loc[39+i:538+i,['close','SMA_7',
       'SMA_14', 'SMA_21', 'RSI_7', 'RSI_14', 'RSI_21', 'ATR_14', 'bb_mavg',
       'bb_hband', 'bb_lband', 'H-L', 'H-PC', 'L-PC', 'TR', '+DM', '-DM',
       '+DMMA', '-DMMA', 'TRMA', '+DI', '-DI', 'DX', 'ADX', 'RSI_indicator',
       'DI_indicator']]
  label=label_train[i:i+500]
  model.fit(df_train,label,epochs=4)

ImportError: ignored

In [ ]:
final=[]

for i in range(0,len(label_train)):
  df_train=df.loc[i:i+1,['close','SMA_7',
       'SMA_14', 'SMA_21', 'RSI_7', 'RSI_14', 'RSI_21', 'ATR_14', 'bb_mavg',
       'bb_hband', 'bb_lband', 'H-L', 'H-PC', 'L-PC', 'TR', '+DM', '-DM',
       '+DMMA', '-DMMA', 'TRMA', '+DI', '-DI', 'DX', 'ADX', 'RSI_indicator',
       'DI_indicator']]
  df_train=df_train.to_numpy()
  final.append(model.predict(df_train))
final

KeyboardInterrupt: ignored

In [ ]:
model.save('Windowed.h5')

In [ ]:

m=model.predict(df_train)

In [ ]:
model.fit(df_train,label_train,epochs=7)

Epoch 1/7
796/796 [==============================] - 28s 34ms/step - loss: 0.4906 - mean_squared_error: 0.3694 - mean_absolute_percentage_error: 384295872.0000
Epoch 2/7
408/796 [==============>...............] - ETA: 15s - loss: 0.4890 - mean_squared_error: 0.3795 - mean_absolute_percentage_error: 392435712.0000

KeyboardInterrupt: ignored

In [ ]:
model.fit(df_train,epochs=30)

Epoch 1/30


ValueError: ignored

In [ ]:
model.fit(df_train,label_train,epochs=30)

Epoch 1/30
796/796 [==============================] - 20s 25ms/step - loss: 0.4911 - mean_squared_error: 0.4500 - mean_absolute_percentage_error: 468199456.0000
Epoch 2/30
796/796 [==============================] - 20s 25ms/step - loss: 0.4914 - mean_squared_error: 0.4527 - mean_absolute_percentage_error: 468718304.0000
Epoch 3/30
796/796 [==============================] - 20s 25ms/step - loss: 0.4911 - mean_squared_error: 0.4530 - mean_absolute_percentage_error: 469074144.0000
Epoch 4/30
796/796 [==============================] - 19s 24ms/step - loss: 0.4910 - mean_squared_error: 0.4594 - mean_absolute_percentage_error: 471780128.0000
Epoch 5/30
796/796 [==============================] - 19s 24ms/step - loss: 0.4907 - mean_squared_error: 0.4681 - mean_absolute_percentage_error: 475910016.0000
Epoch 6/30
796/796 [==============================] - 20s 25ms/step - loss: 0.4908 - mean_squared_error: 0.4469 - mean_absolute_percentage_error: 462585312.0000
Epoch 7/30
796/796 [==============

In [ ]:
import math
df["technical wonly"]=0.5
for i in range(20):
  df["technical wonly"][i]=0.5
for j in range(20,10316):  
  if (df['RSI_7'][j]>60 and df['+DI'][j]>df['-DI'][j]):
    df["technical wonly"][j]=1
  elif df['RSI_7'][j]<50 and df['+DI'][j]>df['-DI'][j]:
    df["technical wonly"][j]=0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
difference=df["technical wonly"]-df3['target']
difference=difference.mod(*)
difference.mean()

SyntaxError: ignored

In [ ]:
df2_test=df2.loc[:,['RSI_7', 'RSI_14', 'RSI_21', 'ATR_14', 'bb_mavg',
       'bb_hband', 'bb_lband', 'open', 'high', 'low', 'close']]
df2_test=df2_test.to_numpy()
len(df2_test)

In [ ]:
from sklearn.metrics import roc_auc_score


In [ ]:
roc_auc_score(label_train,m)